In [2]:
import numpy as np
import bilby
from sympy import sin,cos,log,sqrt
from astropy import constants as const
import matplotlib.pyplot as plt

# import sys
# import os
# import six

OrbitRadiusInS = 1e8 /const.c.value    # 1e5 km
MearthInS      = const.M_earth.value*const.G.value/const.c.value**3
OrbitPeriodInS = 2*np.pi*np.sqrt(OrbitRadiusInS**3/MearthInS)
YearInS  = 31536000.0   # one year in [s]

c = 299792458.0  # m/s
G = 6.67*1.0e-11 
AU = 1.5e+11
parsec = 3.085677581491367e+16  # m
solar_mass = 1.9885469549614615e+30  # Kg

def InspiralWaveform(t, m1, m2, DL, Tc, iota, phic, psi, thetaS, phiS):
    c = 299792458.0  # m/s
    G = 6.67*1.0e-11 
    AU = 1.5e11
    fm = 1.0/YearInS
    eta = m1*m2/(m1+m2)**2  # symmetric mass ratio 
    m = m1+m2     # total mass
    tau = c**3*eta/(5*G*m)*(Tc-t);
    wt = c**3/(8*G*m)*(tau**(-3/8)+(743/2688+11/32*eta)*tau**(-5/8)-3*np.pi/10*tau**(-3/4)\
        +(1855099/14450688+56975/258048*eta+371/2048*eta**2)*tau**(-7/8))
    PHI = -2/eta*(tau**(5/8)+(3715/8064+55/96*eta)*tau**(3/8)-3*np.pi/4*tau**(1/4)\
        +(9275495/14450688+284875/258048*eta+1855/2048*eta**2)*tau**(1/8))\
        +wt*AU/c*sin(thetaS)*cos(2*np.pi*fm*t-phiS)      # phi_Doppler
    x = (G*m*wt/c**3)**(2/3)
    hplus = 2*G*m*eta/(c**2*DL)*(1+cos(iota)**2)*x*cos(PHI+phic)
    hcros = -4*G*m*eta/(c**2*DL)*cos(iota)*x*sin(PHI+phic)
    return hplus, hcros

def Dplus_TQ(t,thetaS,phiS):
    """For TianQin (one Michelson interferometer): (thetaS,phiS) is location of source,
    (thJ,phJ) is latitude and longitude of J0806 in heliocentric-ecliptic frame"""
    thJ  = 1.65273
    phJ  = 2.10213
    kap = 2*np.pi/OrbitPeriodInS* t
#    kap = 2*np.pi/YearInS* t(f,theta)
    return np.sqrt(3.)/32*(8*cos(2*kap) *((3 + cos(2*thetaS)) *sin(2*(phiS - phJ))*  
            cos(thJ) + 2*sin(thJ) *sin(phiS - phJ)*sin(2*thetaS))- 2*sin(2*kap)* (3 +               
            cos(2*(phiS - phJ))*(9 + cos(2*thetaS)*(3 + cos(2*thJ))) -6 *cos(2*thJ)*(sin(phiS - phJ))**2 -               
            6* cos(2*thetaS)*(sin(thJ))**2 + 4*cos(phiS - phJ)*sin(2*thJ)*sin(2*thetaS))) 

def Dcros_TQ(t,thetaS,phiS):
    """For TianQin (one Michelson interferometer): (thetaS,phiS) is location of source, 
    (thJ,phJ) is latitude and longitude of J0806 in heliocentric-ecliptic frame"""
    thJ  = 1.65273
    phJ  = 2.10213
    kap = 2*np.pi/OrbitPeriodInS* t
#    kap = 2*np.pi/YearInS* t(f,theta)
    return np.sqrt(3.)/4*(-4*cos(2*kap)*(cos(2*(phiS-phJ))*cos(thJ)*cos(thetaS)+                 
            cos(phiS-phJ)*sin(thetaS)*sin(thJ))+sin(2*kap)*(cos(thetaS)*(3+cos(2*thJ))*sin(2*(phJ-phiS))+                
            2*sin(phJ-phiS)*sin(thetaS)*sin(2*thJ)))

def Fplus_TQ(t,thetaS,phiS,psi):
    """antenna pattern function for '+' mode"""
    return (cos(2*psi)*Dplus_TQ(t,thetaS,phiS)-sin(2*psi)*Dcros_TQ(t,thetaS,phiS))/2.

def Fcros_TQ(t,thetaS,phiS,psi):
    """antenna pattern function for '×' mode"""
    return (sin(2*psi)*Dplus_TQ(t,thetaS,phiS)+cos(2*psi)*Dcros_TQ(t,thetaS,phiS))/2.


In [3]:
def strain_mbhb(t,m1, m2, DL, Tc, iota, phic, psi, thetaS, phiS):
    Fplus = []
    Fcros = []
    hplus = []
    hcros = []
    
    for i in t:
        Fp = Fplus_TQ(i,thetaS,phiS,psi)
        Fc = Fcros_TQ(i,thetaS,phiS,psi)
        Fplus.append(Fp)
        Fcros.append(Fc)
        
        hp, hc = InspiralWaveform(i,m1, m2, DL, Tc, iota, phic, psi, thetaS, phiS)
        hplus.append(hp)
        hcros.append(hc)
        
    return np.array(Fplus)*np.array(hplus) + np.array(Fcros)*np.array(hcros)

In [5]:
m1 = 1.0e6*solar_mass
m2 = 2.0e5*solar_mass
Tc = 1.04*YearInS
DL = 11.01*1.0e9*parsec


thetaS = 1.6398
phiS = 4.1226
psi = 1.373
iota = 0.7647
phic = 0.9849

sigma = 1.0e-21

duration = 1638400
time = np.arange(0, duration, 86400)

data = strain_mbhb(time,m1, m2, DL, Tc, iota, phic, psi, thetaS, phiS) + sigma*np.random.randn(len(time))
print(data)

[-2.44524620183316e-21 8.74930394895585e-21 8.84818392874461e-21
 -2.74604748650525e-21 -1.55443818443420e-20
 -5.27838603130760e-21 1.14892169650310e-20 8.99782179670935e-21
 -5.85445663632213e-21 -7.93005185182193e-21
 7.72523060358184e-21 5.98679453980629e-21 -1.08303550404910e-20
 -5.28794851021857e-21 1.41334618043666e-20 2.40538862226611e-21
 -1.27704781300611e-20 2.49425424998932e-21 1.04978405462607e-20]


# F-Statistic method

In [6]:
m_fs = m1+m2
eta_fs = m1*m2/(m1+m2)**2
thetaS_fs = thetaS
phiS_fs = phiS
Tc_fs = Tc

def Fstatistic(t,m_fs,eta_fs,thetaS_fs,phiS_fs,Tc_fs):
    c = 299792458.0  # m/s
    G = 6.67*1.0e-11 
    AU = 1.5e11
    fm = 1.0/YearInS
    tau = c**3*eta_fs/(5*G*m_fs)*(Tc_fs-t)
    wt = c**3/(8*G*m_fs)*(tau**(-3/8)+(743/2688+11/32*eta_fs)*tau**(-5/8)-3*np.pi/10*tau**(-3/4)\
        +(1855099/14450688+56975/258048*eta_fs+371/2048*eta_fs**2)*tau**(-7/8))
    PHI = -2/eta_fs*(tau**(5/8)+(3715/8064+55/96*eta_fs)*tau**(3/8)-3*np.pi/4*tau**(1/4)\
        +(9275495/14450688+284875/258048*eta_fs+1855/2048*eta_fs**2)*tau**(1/8))\
        +wt*AU/c*sin(thetaS_fs)*cos(2*np.pi*fm*t-phiS_fs)      # phi_Doppler
    x = (G*m_fs*wt/c**3)**(2/3)
    mo = G*m_fs*eta_fs/c**3*x
    A1 = mo*Dplus_TQ(t,thetaS,phiS)*cos(PHI)
    A2 = mo*Dcros_TQ(t,thetaS,phiS)*cos(PHI)
    A3 = mo*Dplus_TQ(t,thetaS,phiS)*sin(PHI)
    A4 = mo*Dcros_TQ(t,thetaS,phiS)*sin(PHI)
    
    return A1,A2,A3,A4
    

In [8]:
A1_1 = []
A2_1 = []
A3_1 = []
A4_1 = []
for i in time:
    A1,A2,A3,A4 = Fstatistic(i,m_fs,eta_fs,thetaS_fs,phiS_fs,Tc_fs)
    A1_1.append(A1)
    A2_1.append(A2)
    A3_1.append(A3)
    A4_1.append(A4)

A1_1 = np.array(A1_1)
A2_1 = np.array(A2_1)
A3_1 = np.array(A3_1)
A4_1 = np.array(A4_1)

A_matrix = np.array([A1_1,A2_1,A3_1,A4_1])

M=np.zeros([4,4])
N=np.zeros([4,1])
a=np.zeros([4,1])

for k in np.arange(0,4):
    for j in np.arange(0,4):
        M[k][j] = np.vdot(A_matrix[k,:],A_matrix[j,:])

for l in np.arange(0,4):
    N[l][0]=np.vdot(A_matrix[l,:],np.transpose(data))

for p in np.arange(0,4):
    inverse_M = np.linalg.inv(M)
    a[p][0] =np.vdot(inverse_M[p,:], N[:,0])
    
LOG_L = 1/2*np.vdot(N[:,0],np.transpose(a[:,0]))/sigma**2
print(LOG_L)

732.9618434561618


In [9]:
print(a)

[[-1.11065496e-18]
 [ 7.18224844e-19]
 [ 8.02376642e-19]
 [ 1.00318615e-18]]


In [10]:
a1=a[0]
a2=a[1]
a3=a[2]
a4=a[3]

Ap = ((a1+a4)**2+(a2-a3)**2)**(1/2)+((a1-a4)**2+(a2+a3)**2)**(1/2);
Ac = ((a1+a4)**2+(a2-a3)**2)**(1/2)-((a1-a4)**2+(a2+a3)**2)**(1/2);
AA = (Ap+(Ap**2-Ac**2)**(1/2));


# recover Extrinsic parameter
iota_est = np.arccos(-Ac/(AA))
psi_est = 1/2*np.arctan((Ap*a4-Ac*a1)/(-(Ac*a2+Ap*a3)))+np.pi/2
cc = np.sign(sin(2*psi))
DL_est = 4*c/AA
phic_est = np.arctan(cc*(Ap*a4-Ac*a1)/(-cc*(Ap*a2+Ac*a3)))

print(iota_est,psi_est,DL_est/parsec,phic_est)

[0.89258172] [1.56061824] [9.88162162e+09] [0.64395409]


# LOG_Likelihood

In [11]:
def LOG_Likelihood(time,m_fs,eta_fs,thetaS_fs,phiS_fs,Tc_fs):
    sigma = 1.0e-21
    A1_1 = []
    A2_1 = []
    A3_1 = []
    A4_1 = []
    for i in time:
        A1,A2,A3,A4 = Fstatistic(i,m_fs,eta_fs,thetaS_fs,phiS_fs,Tc_fs)
        A1_1.append(A1)
        A2_1.append(A2)
        A3_1.append(A3)
        A4_1.append(A4)

    A1_1 = np.array(A1_1)
    A2_1 = np.array(A2_1)
    A3_1 = np.array(A3_1)
    A4_1 = np.array(A4_1)

    A_matrix = np.array([A1_1,A2_1,A3_1,A4_1])

    M=np.zeros([4,4])
    N=np.zeros([4,1])
    a=np.zeros([4,1])

    for k in np.arange(0,4):
        for j in np.arange(0,4):
            M[k][j] = np.vdot(A_matrix[k,:],A_matrix[j,:])

    for l in np.arange(0,4):
        N[l][0]=np.vdot(A_matrix[l,:],np.transpose(strain))

    for p in np.arange(0,4):
        inverse_M = np.linalg.inv(M)
        a[p][0] =np.vdot(inverse_M[p,:], N[:,0])
    
    LOG_L = 1/2*np.vdot(N[:,0],np.transpose(a[:,0]))/sigma**2
    return LOG_L

In [15]:
from cpnest.model import Model

class FS_time_TQ(Model):
    names = ['m_fs','eta_fs','thetaS_fs','phiS_fs','Tc_fs'] # parameter names (this is a required variables for the class)

    # define the boundaries using for initial sampling of the live points
    m_fsmin = 1.0e6*solar_mass  # lower range on c (the same as the uniform c prior lower bound)
    m_fsmax = 2.0e6*solar_mass   # upper range on c (the same as the uniform c prior upper bound)

    eta_fsmin = 0.0     
    eta_fsmax = 0.5 
    
    thetaS_fsmin = 0.0
    thetaS_fsmax = np.pi
    
    phiS_fsmin = 0.0
    phiS_fsmax = 2*np.pi
    
    Tc_fsmin = 0.9*YearInS
    Tc_fsmax = 1.2*YearInS
    
    m_fsbounds = [m_fsmin, m_fsmax]
    eta_fsbounds = [eta_fsmin, eta_fsmax]
    thetaS_fsbounds = [thetaS_fsmin, thetaS_fsmax]
    phiS_fsbounds = [phiS_fsmin, phiS_fsmax]
    Tc_fsbounds = [Tc_fsmin, Tc_fsmax]

    # NOTE: the bounds could instead be set through arguments passed to the __init__
    # function for the class if wanted
    bounds=[m_fsbounds, eta_fsbounds, thetaS_fsbounds, phiS_fsbounds, Tc_fsbounds] # upper and lower bounds on each parameter (required for the class)

    def __init__(self, data, time, modelfunc, sigma):
        # set the data
        self._data = data         # oberserved data
        self._ndata = len(data)   # number of data points
        self._model = modelfunc      # model function
        self._sigma = sigma

    def LOG_Likelihood(self,params):
        # extract the parameters
        m_fs = params['m_fs']
        eta_fs = params['eta_fs']
        thetaS_fs = params['thetaS_fs']
        phiS_fs = params['phiS_fs']
        Tc_fs = params['Tc_fs']
        
        sigma = 1.0e-21
        A1_1 = []
        A2_1 = []
        A3_1 = []
        A4_1 = []
        for i in self.time:
            A1,A2,A3,A4 = Fstatistic(i,m_fs,eta_fs,thetaS_fs,phiS_fs,Tc_fs)
            A1_1.append(A1)
            A2_1.append(A2)
            A3_1.append(A3)
            A4_1.append(A4)

        A1_1 = np.array(A1_1)
        A2_1 = np.array(A2_1)
        A3_1 = np.array(A3_1)
        A4_1 = np.array(A4_1)

        A_matrix = np.array([A1_1,A2_1,A3_1,A4_1])

        M=np.zeros([4,4])
        N=np.zeros([4,1])
        a=np.zeros([4,1])

        for k in np.arange(0,4):
            for j in np.arange(0,4):
                M[k][j] = np.vdot(A_matrix[k,:],A_matrix[j,:])

        for l in np.arange(0,4):
            N[l][0]=np.vdot(A_matrix[l,:],np.transpose(self.strain))

        for p in np.arange(0,4):
            inverse_M = np.linalg.inv(M)
            a[p][0] =np.vdot(inverse_M[p,:], N[:,0])
    
        return 1/2*np.vdot(N[:,0],a[:,0])/sigma**2

#     def log_prior(self,p):

#         if not self.in_bounds(p): return -np.inf # check parameters are all in bounds

#         # Gaussian prior on m
#         lp = 0.
#         m = p['m']
#         lp -= 0.5*((m-self.mmu)/self.msigma)**2 # no need for normalisation constant on the prior

#         return lp

In [ ]:
# import cpnest
import cpnest

print('CPNest version: {}'.format(cpnest.__version__))

nlive = 1024      # number of live point
maxmcmc = 1024    # maximum MCMC chain length
nthreads = 4      # use one CPU core

# set up the algorithm
work = cpnest.CPNest(FS_time_TQ(data, time, LOG_Likelihood, sigma), verbose=0,
                     nthreads=nthreads, nlive=nlive, maxmcmc=maxmcmc);
# run the algorithm
from datetime import datetime
t0 = datetime.now()
work.run();
t1 = datetime.now()

timecpnest = (t1-t0)

CPNest version: 0.9.7
Running with 4 parallel threads


Process Process-2:
Process Process-3:
Traceback (most recent call last):
Process Process-4:
  File "/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Process Process-5:
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/anaconda3/lib/python3.7/site-packages/cpnest/sampler.py", line 155, in produce_sample
    self._produce_sample()
  File "/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/a